<center>
<img src="https://drive.google.com/uc?id=1f1gGVI-rxcHjA90WEGNvvtSXF1pAxQwg" alt="Fasilkom UI" width="300"/>

CSCE604137 • Penambangan Data

Semester Ganjil 2021/2022

Fakultas Ilmu Komputer, Universitas Indonesia

##**Tugas Kelompok 1**

###**Tenggat Waktu: 15 April 2022, 22.00 WIB**
</center>

####**Ketentuan:**

Anggota Kelompok:
- Amanda Carrisa Ashardian - 1906696969
- Muhamad Yoga Mahendra - 1806205256
- Nabila Fathia Zahra - 1706022741


In [6]:
# Import
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as scp
import seaborn as sn
import sklearn as sk
import statsmodels.api as sm
from scipy.linalg import eigh
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import *
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import pairwise, silhouette_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
!pip install PySastrawi

In [7]:
from sklearn.metrics import accuracy_score, f1_score


In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

In [10]:
# Helper Function Definitions
def cek_null(df):
    col_na = df.isnull().sum().sort_values(ascending=False)
    percent = col_na / len(df)
    
    missing_data = pd.concat([col_na, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data[missing_data['Total'] > 0])
  
def count_outliers(df, col):
    outliers = 0
    q1 = df[col].quantile(.25)
    q3 = df[col].quantile(.75)
    iqr = q3 - q1
    w = 1.5 * iqr

    for i in df.index:
        row = df[col][i]
        if (row < (q1 - w) or row > (q3 + w)):
            outliers += 1
    return outliers

TO DO List:
- Preprosesing data (ubah jadi tabular/csv)
- feature exploration (jadiin bow/word embedding)
- Implement naive bayes/multilayer/smth
- cek hasil pake scoring macro micro F1 scoring, hamming loss, exact match ratio
- pake supervised learning 
- abis itu ensembled (N model digabung/agregat terus divote yang mana yg paling banyak)

Pembagian:

Amanda:
macro micro F1 scoring, hamming loss

Thia:
Preprosesing data,
feature exploration

Yoga:
exact match ratio,
bantu manda thia


In [11]:
# Converting csv to dataframe

# contoh = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/contoh_data.csv')
# human_ant = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/human_annotated.csv')
# machine_ant = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/machine_annotated.csv')
# testing = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/testing_data.csv')
# unlabeled = pd.read_csv('/content/drive/MyDrive/Data TK 1 Text Mining/clean/unlabeled_data.csv')

contoh = pd.read_csv('contoh_data.csv')
human_ant = pd.read_csv('human_annotated.csv')
machine_ant = pd.read_csv('machine_annotated.csv')
testing = pd.read_csv('testing_data.csv')
unlabeled = pd.read_csv('unlabeled_data.csv')


In [12]:
machine_ant.append(human_ant, ignore_index = True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,Unnamed: 4,Unnamed: 5,id,isi,judul,label_0,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8
0,NaN,NaN,AD-10,dok saya cowok ko alat kelamin saya gatal ya ....,gatal-gatal dan bintik merah di kelamin,Kesehatan Kulit dan Kelamin,Umum,Urologi,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,AD-15,selamat siang dok . sudah beberapa minggu ini ...,sering buang air kecil,Bedah,Urologi,Penyakit Dalam,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,AD-28,assalammu alaikum . salam sejahtera bagi kita ...,penyebab nyeri dada,Jantung dan Pembuluh Darah,Penyakit Dalam,Umum,Bedah,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,AD-33,"malam dok , tadi saya digigit anjing tetapi an...",penanganan luka digigit anjing,Penyakit Dalam,Saraf,Kesehatan Anak,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,AD-36,"malam dok , sebelum nya knlin dok nma saya riw...",insomnia karena banyak pikiran,Kesehatan Jiwa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13296,NaN,NaN,KD-43177,"malam dok , saya ingin tahu apakah penyakit us...",usus buntu,Umum,Bedah,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13297,NaN,NaN,KD-43189,selamat siang dokter saya mau konsultasi soal ...,telinga berdenging,Umum,"Kesehatan Telinga, Hidung dan Tenggorokan (THT)",NaN,NaN,NaN,NaN,NaN,NaN,NaN
13298,NaN,NaN,KD-43353,salam sehat dok untuk menjaga kesehatan selain...,aktivitas berat ?,Umum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13299,NaN,NaN,KD-43435,"dok , saya mau tanya seputar penyakit liver.ka...",penyakit liver,Kesehatan Gizi,Penyakit Dalam,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
machine_ant["isi"] = machine_ant["judul"] + machine_ant["isi"]

In [14]:
KebidananArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kebidanan dan Kandungan" or row['label_1'] == "Kebidanan dan Kandungan" or row['label_2'] == "Kebidanan dan Kandungan"or row['label_3'] == "Kebidanan dan Kandungan"or row['label_4'] == "Kebidanan dan Kandungan"or row['label_5'] == "Kebidanan dan Kandungan"or row['label_6'] == "Kebidanan dan Kandungan"or row['label_7'] == "Kebidanan dan Kandungan"or row['label_8'] == "Kebidanan dan Kandungan":
    KebidananArray.append("1")
  else:
    KebidananArray.append("0")
print(len(KebidananArray))



11526


In [15]:
DalamArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Penyakit Dalam" or row['label_1'] == "Penyakit Dalam" or row['label_2'] == "Penyakit Dalam" or row['label_3'] == "Penyakit Dalam" or row['label_4'] == "Penyakit Dalam" or row['label_5'] == "Penyakit Dalam" or row['label_6'] == "Penyakit Dalam" or row['label_7'] == "Penyakit Dalam" or row['label_8'] == "Penyakit Dalam":
    DalamArray.append("1")
  else:
    DalamArray.append("0")
print(len(DalamArray))


11526


In [16]:
AnakArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kesehatan Anak" or row['label_1'] == "Kesehatan Anak" or row['label_2'] == "Kesehatan Anak" or row['label_3'] == "Kesehatan Anak" or row['label_4'] == "Kesehatan Anak" or row['label_5'] == "Kesehatan Anak" or row['label_6'] == "Kesehatan Anak" or row['label_7'] == "Kesehatan Anak" or row['label_8'] == "Kesehatan Anak":
    AnakArray.append("1")
  else:
    AnakArray.append("0")
print(len(AnakArray))
print(AnakArray[0])


11526
0


In [17]:
KulitArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kesehatan Kulit dan Kelamin" or row['label_1'] == "Kesehatan Kulit dan Kelamin" or row['label_2'] == "Kesehatan Kulit dan Kelamin" or row['label_3'] == "Kesehatan Kulit dan Kelamin" or row['label_4'] == "Kesehatan Kulit dan Kelamin" or row['label_5'] == "Kesehatan Kulit dan Kelamin"or row['label_6'] == "Kesehatan Kulit dan Kelamin" or row['label_7'] == "Kesehatan Kulit dan Kelamin" or row['label_8'] == "Kesehatan Kulit dan Kelamin":
    KulitArray.append("1")
  else:
    KulitArray.append("0")
print(len(KulitArray))
print(KulitArray[0])


11526
1


In [18]:
GiziArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kesehatan Gizi" or row['label_1'] == "Kesehatan Gizi" or row['label_2'] == "Kesehatan Gizi"or row['label_3'] == "Kesehatan Gizi" or row['label_4'] == "Kesehatan Gizi"or row['label_5'] == "Kesehatan Gizi" or row['label_6'] == "Kesehatan Gizi"or row['label_7'] == "Kesehatan Gizi" or row['label_8'] == "Kesehatan Gizi":
    GiziArray.append("1")
  else:
    GiziArray.append("0")
print(len(GiziArray)) 

11526


In [19]:
THTArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_1'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_2'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)"or row['label_3'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_4'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)"or row['label_5'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_6'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)"or row['label_7'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)" or row['label_8'] == "Kesehatan Telinga, Hidung dan Tenggorokan (THT)":
    THTArray.append("1")
  else:
    THTArray.append("0")
print(len(THTArray)) 

11526


In [20]:
GigiArray = []

for index, row in machine_ant.iterrows():
  if row['label_0'] == "Gigi" or row['label_1'] == "Gigi" or row['label_2'] == "Gigi"or row['label_3'] == "Gigi" or row['label_4'] == "Gigi"or row['label_5'] == "Gigi" or row['label_6'] == "Gigi"or row['label_7'] == "Gigi" or row['label_8'] == "Gigi":
    GigiArray.append("1")
  else:
    GigiArray.append("0")
print(len(GigiArray)) 

11526


In [21]:
MataArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kesehatan Mata" or row['label_1'] == "Kesehatan Mata" or row['label_2'] == "Kesehatan Mata"or row['label_3'] == "Kesehatan Mata" or row['label_4'] == "Kesehatan Mata"or row['label_5'] == "Kesehatan Mata" or row['label_6'] == "Kesehatan Mata"or row['label_7'] == "Kesehatan Mata" or row['label_8'] == "Kesehatan Mata":
    MataArray.append("1")
  else:
    MataArray.append("0")
print(len(MataArray)) 

11526


In [22]:
BedahArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Bedah" or row['label_1'] == "Bedah" or row['label_2'] == "Bedah"or row['label_3'] == "Bedah" or row['label_4'] == "Bedah"or row['label_5'] == "Bedah" or row['label_6'] == "Bedah"or row['label_7'] == "Bedah" or row['label_8'] == "Bedah":
    BedahArray.append("1")
  else:
    BedahArray.append("0")
print(len(BedahArray)) 

11526


In [23]:
KesehatanJiwaArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Kesehatan Jiwa" or row['label_1'] == "Kesehatan Jiwa" or row['label_2'] == "Kesehatan Jiwa"or row['label_3'] == "Kesehatan Jiwa" or row['label_4'] == "Kesehatan Jiwa"or row['label_5'] == "Kesehatan Jiwa" or row['label_6'] == "Kesehatan Jiwa"or row['label_7'] == "Kesehatan Jiwa" or row['label_8'] == "Kesehatan Jiwa":
    KesehatanJiwaArray.append("1")
  else:
    KesehatanJiwaArray.append("0")
print(len(KesehatanJiwaArray)) 

11526


In [24]:
TulangArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Ortopedi (Tulang)" or row['label_1'] == "Ortopedi (Tulang)" or row['label_2'] == "Ortopedi (Tulang)"or row['label_3'] == "Ortopedi (Tulang)" or row['label_4'] == "Ortopedi (Tulang)"or row['label_5'] == "Ortopedi (Tulang)" or row['label_6'] == "Ortopedi (Tulang)"or row['label_7'] == "Ortopedi (Tulang)" or row['label_8'] == "Ortopedi (Tulang)":
    TulangArray.append("1")
  else:
    TulangArray.append("0")
print(len(TulangArray)) 

11526


In [25]:
JantungArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Jantung dan Pembuluh Darah" or row['label_1'] == "Jantung dan Pembuluh Darah" or row['label_2'] == "Jantung dan Pembuluh Darah" or row['label_3'] == "Jantung dan Pembuluh Darah" or row['label_4'] == "Jantung dan Pembuluh Darah" or row['label_5'] == "Jantung dan Pembuluh Darah" or row['label_6'] == "Jantung dan Pembuluh Darah" or row['label_7'] == "Jantung dan Pembuluh Darah" or row['label_8'] == "Jantung dan Pembuluh Darah":
    JantungArray.append("1")
  else:
    JantungArray.append("0")
print(len(JantungArray)) 

11526


In [26]:
UrologiArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Urologi" or row['label_1'] == "Urologi" or row['label_2'] == "Urologi"or row['label_3'] == "Urologi" or row['label_4'] == "Urologi"or row['label_5'] == "Urologi" or row['label_6'] == "Urologi"or row['label_7'] == "Urologi" or row['label_8'] == "Urologi":
    UrologiArray.append("1")
  else:
    UrologiArray.append("0")
print(len(UrologiArray))

11526


In [27]:
SarafArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Saraf" or row['label_1'] == "Saraf" or row['label_2'] == "Saraf"or row['label_3'] == "Saraf" or row['label_4'] == "Saraf"or row['label_5'] == "Saraf" or row['label_6'] == "Saraf"or row['label_7'] == "Saraf" or row['label_8'] == "Saraf":
    SarafArray.append("1")
  else:
    SarafArray.append("0")
print(len(SarafArray)) 

11526


In [28]:

ParuArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Pulmonologi (Paru)" or row['label_1'] == "Pulmonologi (Paru)" or row['label_2'] == "Pulmonologi (Paru)"or row['label_3'] == "Pulmonologi (Paru)" or row['label_4'] == "Pulmonologi (Paru)"or row['label_5'] == "Pulmonologi (Paru)" or row['label_6'] == "Pulmonologi (Paru)"or row['label_7'] == "Pulmonologi (Paru)" or row['label_8'] == "Pulmonologi (Paru)":
    ParuArray.append("1")
  else:
    ParuArray.append("0")
print(len(ParuArray))

11526


In [29]:
UmumArray = []
for index, row in machine_ant.iterrows():
  if row['label_0'] == "Umum" or row['label_1'] == "Umum" or row['label_2'] == "Umum":
    UmumArray.append("1")
  else:
    UmumArray.append("0")
print(len(UmumArray))

11526


In [30]:
#masukin ke tabel machine_ant hasil encoding (BELOM DI JADIIN MODEL)
y = [KebidananArray, DalamArray, AnakArray, KulitArray, GiziArray, THTArray, GigiArray, MataArray, BedahArray, KesehatanJiwaArray, TulangArray, JantungArray, UrologiArray, SarafArray, ParuArray, UmumArray]

machine_ant["KebidananArray"] = KebidananArray
machine_ant["DalamArray"] = DalamArray
machine_ant["AnakArray"] = AnakArray
machine_ant["KulitArray"] = KulitArray
machine_ant["GiziArray"] = GiziArray
machine_ant["THTArray"] = THTArray
machine_ant["GigiArray"] = GigiArray
machine_ant["MataArray"] = MataArray
machine_ant["BedahArray"] = BedahArray
machine_ant["KesehatanJiwaArray"] = KesehatanJiwaArray
machine_ant["TulangArray"] = TulangArray
machine_ant["JantungArray"] = JantungArray
machine_ant["UrologiArray"] = UrologiArray
machine_ant["SarafArray"] = SarafArray
machine_ant["ParuArray"] = ParuArray
machine_ant["UmumArray"] = UmumArray


In [31]:
testing['isi'] = testing['judul']+testing['isi']
testing

,id,judul,isi
0,DS-45,kenapa telinga berdengung tidak bisa hilang ?,kenapa telinga berdengung tidak bisa hilang ?s...
1,DS-339,apabila anak anda lambat berbicara,apabila anak anda lambat berbicarasalam kenal ...
2,AD-18,sakit perut setiap habis makan dan minum,sakit perut setiap habis makan dan minumselama...
3,AD-66,cara mengobati vertigo berkepanjangan,cara mengobati vertigo berkepanjanganselamat m...
4,AD-73,imunisasi dapat dan polio,"imunisasi dapat dan poliomalam dok , anak saya..."
...,...,...,...
2922,KD-43404,keluhan setelah berhubungan,keluhan setelah berhubungandok minggu lalu ber...
2923,KD-43406,ingin menambah tinggi badan,"ingin menambah tinggi badanselamat malam dok ,..."
2924,KD-43411,anak 1 tahun boleh dikasih nasi ?,anak 1 tahun boleh dikasih nasi ?apakah bayi y...
2925,KD-43446,mencegah infeksi virus zika,mencegah infeksi virus zikaselamat siang dok s...


In [32]:
# Normalizing text & removing stopwords
import re

def normalize(text):
    res = text.lower().strip() # Mengubah uppercase menjadi lowercase dan melakukan trimming pada teks
    res = re.sub("\d", "", res) # Menghilangkan angka
    res = re.sub("\s+", " ", res) # Menghilangkan spasi berlebih
    res = re.sub("[^\w\s]", "", res) # Menghilangkan tanda baca
    return res

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

stop_factory = StopWordRemoverFactory()
lst_stopword = stop_factory.get_stop_words()

# Fungsi untuk menghilangkan stopword dari suatu teks

def remove_stopword(text):
    lst_token = text.split(" ")
    res = []
    for token in lst_token:
        if token not in lst_stopword:
            res.append(token)
    return " ".join(res)

def clean_words(arr):
  corpus = []
  for text in arr:
    normalized_text = normalize(text)
    cleaned_text = remove_stopword(normalized_text)
    corpus.append(cleaned_text)
  return corpus

In [33]:
contoh_isi = contoh['isi']
contoh_isi_clean = clean_words(contoh_isi)

In [34]:
for i in range(len(contoh_isi_clean)):
  contoh['isi'][i] = contoh_isi_clean[i]

In [35]:
human_isi = human_ant['isi'].astype(str)
human_isi_clean = clean_words(human_isi)

machine_isi = machine_ant['isi'].astype(str)
machine_isi_clean = clean_words(machine_isi)

testing_isi = testing['isi'].astype(str)
testing_isi_clean = clean_words(testing_isi)

unlabeled_isi = unlabeled['isi'].astype(str)
unlabeled_isi_clean = clean_words(unlabeled_isi)

In [36]:
def replace_with_cleaned(arr,arr_clean):
  for i in range(len(arr_clean)):
    arr[i] = arr_clean[i]

In [37]:
replace_with_cleaned(human_ant['isi'],human_isi_clean)
replace_with_cleaned(machine_ant['isi'],machine_isi_clean)
replace_with_cleaned(testing['isi'],testing_isi_clean)
replace_with_cleaned(unlabeled['isi'],unlabeled_isi_clean)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


**word2vec VECTORIZING**

In [39]:
!python -m pip install -U gensim

     |████████████████████████████████| 24.0MB 76kB/s  eta 0:00:01     |██████████████████████████▊     | 20.1MB 2.5MB/s eta 0:00:02
     |████████████████████████████████| 61kB 2.4MB/s eta 0:00:011


In [40]:
#Word Embedding using Word2Vec

# Class untuk enkapsulasi beberapa fungsionalitas yang ada

from gensim import models

class Word2VecModel:
    #Yoga
    # pretrained_file = "/content/drive/MyDrive/Colab Notebooks/Data Mining/Data TK 1 Text Mining/word2vec.id.300d.txt.gz"
    #Amanda
    pretrained_file = "word2vec.id.300d.txt.gz"
    vector_size = 300
    model = models.KeyedVectors.load_word2vec_format(pretrained_file)

    # Fungsi untuk mencari representasi vektor dari suatu kata  
    def get_word_embedding(self, word):
        try:
            return self.model[word]
        except: # Melakukan handling terhadap OOV
            return np.nan
    
    # Fungsi untuk mencari representasi vektor dari suatu kalimat
    def get_sentence_embedding(self, sent):
        lst_token = re.findall("[^\s]{3,}", sent.lower())
        rep = []
        for token in lst_token:
            try:
                token_rep = self.model[token]
                rep.append(token_rep)
            except: # Melakukan handling terhadap OOV
                continue
        return np.mean(rep, axis=0)
    
    # Fungsi untuk mengubah setiap teks pada suatu corpus menjadi representasi vektornya
    def transform(self, corpus):
        res = []
        for doc in corpus:
            tmp = self.get_sentence_embedding(doc)
            if type(tmp) == np.float64: # Melakukan handling terhadap OOV
                tmp = np.zeros(self.vector_size)
            res.append(tmp)
        return np.array(res)

In [41]:
model_word2vec = Word2VecModel()

In [42]:
#udah ada label
res_human = model_word2vec.transform(human_ant['isi'])
res_machine = model_word2vec.transform(machine_ant['isi'])
res_contoh = model_word2vec.transform(contoh['isi'])

#belom ada label
res_testing = model_word2vec.transform(testing['isi'])
res_unlabeled = model_word2vec.transform(unlabeled['isi'])

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# **model bidan**

In [43]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_bidan, test_corpus_bidan, train_label_bidan, test_label_bidan = train_test_split(
    res_machine, 
    machine_ant['KebidananArray'], 
    test_size=0.3,
    random_state=2022
)


In [44]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import GaussianNB
 

classifier_svc = GaussianNB()
classifier_svc_bidan = classifier_svc.fit(train_corpus_bidan, train_label_bidan)
#langsung ke sini kalo testing dataset
pred_res_bidan = classifier_svc_bidan.predict(test_corpus_bidan)


# **model penyakit dalam**

In [45]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_dalam, test_corpus_dalam, train_label_dalam, test_label_dalam = train_test_split(
    res_machine, 
    machine_ant['DalamArray'], 
    test_size=0.3,
    random_state=2022
)


In [46]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_dalam = classifier_svc.fit(train_corpus_dalam, train_label_dalam)
#langsung ke sini kalo testing dataset
pred_res_dalam = classifier_svc_dalam.predict(test_corpus_dalam)


# **model anak**

In [47]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_anak, test_corpus_anak, train_label_anak, test_label_anak = train_test_split(
    res_machine, 
    machine_ant['AnakArray'], 
    test_size=0.3,
    random_state=2022
)


In [48]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_anak = classifier_svc.fit(train_corpus_anak, train_label_anak)
#langsung ke sini kalo testing dataset
pred_res_anak = classifier_svc_anak.predict(test_corpus_anak)


# **model kulit kelamin**

In [49]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_kulit, test_corpus_kulit, train_label_kulit, test_label_kulit = train_test_split(
    res_machine, 
    machine_ant['KulitArray'], 
    test_size=0.3,
    random_state=2022
)


In [50]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_kulit = classifier_svc.fit(train_corpus_kulit, train_label_kulit)
#langsung ke sini kalo testing dataset
pred_res_kulit = classifier_svc_kulit.predict(test_corpus_kulit)


# **model gizi**

In [51]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_gizi, test_corpus_gizi, train_label_gizi, test_label_gizi = train_test_split(
    res_machine, 
    machine_ant['GiziArray'], 
    test_size=0.3,
    random_state=2022
)


In [52]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_gizi = classifier_svc.fit(train_corpus_gizi, train_label_gizi)
#langsung ke sini kalo testing dataset
pred_res_gizi = classifier_svc_gizi.predict(test_corpus_gizi)


# **model THT**

In [53]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_THT, test_corpus_THT, train_label_THT, test_label_THT = train_test_split(
    res_machine, 
    machine_ant['THTArray'], 
    test_size=0.3,
    random_state=2022
)


In [54]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_THT = classifier_svc.fit(train_corpus_THT, train_label_THT)
#langsung ke sini kalo testing dataset
pred_res_THT = classifier_svc_THT.predict(test_corpus_THT)


# **model Gigi**

In [55]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_gigi, test_corpus_gigi, train_label_gigi, test_label_gigi = train_test_split(
    res_machine, 
    machine_ant['GigiArray'], 
    test_size=0.3,
    random_state=2022
)


In [56]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_gigi = classifier_svc.fit(train_corpus_gigi, train_label_gigi)
#langsung ke sini kalo testing dataset
pred_res_gigi = classifier_svc_gigi.predict(test_corpus_gigi)


# **model Mata**

In [57]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_mata, test_corpus_mata, train_label_mata, test_label_mata = train_test_split(
    res_machine, 
    machine_ant['MataArray'], 
    test_size=0.3,
    random_state=2022
)


In [58]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_mata = classifier_svc.fit(train_corpus_mata, train_label_mata)
#langsung ke sini kalo testing dataset
pred_res_mata = classifier_svc_mata.predict(test_corpus_mata)


# **model Bedah**

In [59]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_bedah, test_corpus_bedah, train_label_bedah, test_label_bedah = train_test_split(
    res_machine, 
    machine_ant['BedahArray'], 
    test_size=0.3,
    random_state=2022
)


In [60]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_bedah = classifier_svc.fit(train_corpus_bedah, train_label_bedah)
#langsung ke sini kalo testing dataset
pred_res_bedah = classifier_svc_bedah.predict(test_corpus_bedah)


# **model Kesehatan Jiwa**

In [61]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_jiwa, test_corpus_jiwa, train_label_jiwa, test_label_jiwa = train_test_split(
    res_machine, 
    machine_ant['KesehatanJiwaArray'], 
    test_size=0.3,
    random_state=2022
)


In [62]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_jiwa = classifier_svc.fit(train_corpus_jiwa, train_label_jiwa)
#langsung ke sini kalo testing dataset
pred_res_jiwa = classifier_svc_jiwa.predict(test_corpus_jiwa)


# **model Tulang**

In [63]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_tulang, test_corpus_tulang, train_label_tulang, test_label_tulang = train_test_split(
    res_machine, 
    machine_ant['TulangArray'], 
    test_size=0.3,
    random_state=2022
)


In [64]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_tulang = classifier_svc.fit(train_corpus_tulang, train_label_tulang)
#langsung ke sini kalo testing dataset
pred_res_tulang = classifier_svc_tulang.predict(test_corpus_tulang)


# **model Jantung**

In [65]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_jantung, test_corpus_jantung, train_label_jantung, test_label_jantung = train_test_split(
    res_machine, 
    machine_ant['JantungArray'], 
    test_size=0.3,
    random_state=2022
)


In [66]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_jantung = classifier_svc.fit(train_corpus_jantung, train_label_jantung)
#langsung ke sini kalo testing dataset
pred_res_jantung = classifier_svc_jantung.predict(test_corpus_jantung)


# **model Urologi**

In [67]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_urologi, test_corpus_urologi, train_label_urologi, test_label_urologi = train_test_split(
    res_machine, 
    machine_ant['UrologiArray'], 
    test_size=0.3,
    random_state=2022
)


In [68]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_urologi = classifier_svc.fit(train_corpus_urologi, train_label_urologi)
#langsung ke sini kalo testing dataset
pred_res_urologi = classifier_svc_urologi.predict(test_corpus_urologi)


# **model Saraf**

In [69]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_saraf, test_corpus_saraf, train_label_saraf, test_label_saraf = train_test_split(
    res_machine, 
    machine_ant['SarafArray'], 
    test_size=0.3,
    random_state=2022
)


In [70]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_saraf = classifier_svc.fit(train_corpus_saraf, train_label_saraf)
#langsung ke sini kalo testing dataset
pred_res_saraf = classifier_svc_saraf.predict(test_corpus_saraf)


# **model Paru**

In [71]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_paru, test_corpus_paru, train_label_paru, test_label_paru = train_test_split(
    res_machine, 
    machine_ant['ParuArray'], 
    test_size=0.3,
    random_state=2022
)


In [73]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_paru = classifier_svc.fit(train_corpus_paru, train_label_paru)
#langsung ke sini kalo testing dataset
pred_res_paru = classifier_svc_paru.predict(test_corpus_paru)


# **model Umum**

In [74]:
# Membagi dataset yang ada menjadi 2 bagian, train serta test

train_corpus_umum, test_corpus_umum, train_label_umum, test_label_umum = train_test_split(
    res_machine, 
    machine_ant['UmumArray'], 
    test_size=0.3,
    random_state=2022
)


In [75]:
from sklearn.svm import LinearSVC
from sklearn.metrics import precision_score, f1_score, hamming_loss

classifier_svc = GaussianNB()
classifier_svc_umum = classifier_svc.fit(train_corpus_umum, train_label_umum)
#langsung ke sini kalo testing dataset
pred_res_umum = classifier_svc_umum.predict(test_corpus_umum)


# Finalisasi

URUTAN
1. Kebidanan dan Kandungan
2. Penyakit Dalam
3. Kesehatan Anak
4. Kesehatan Kulit dan Kelamin
5. Kesehatan Gizi
6. Kesehatan Telinga, Hidung dan Tenggorokan (THT)
7. Gigi
8. Kesehatan Mata
9. Bedah
10. Kesehatan Jiwa
11. Ortopedi (Tulang)
12. Jantung dan Pembuluh Darah
13. Urologi
14. Saraf
15. Pulmonologi (Paru)
16. Umum

In [76]:
testing

,id,judul,isi
0,DS-45,kenapa telinga berdengung tidak bisa hilang ?,telinga berdengung hilang selamat sore dokter ...
1,DS-339,apabila anak anda lambat berbicara,anak lambat berbicarasalam kenal team dokterse...
2,AD-18,sakit perut setiap habis makan dan minum,sakit perut habis makan minumselamat malam dok...
3,AD-66,cara mengobati vertigo berkepanjangan,mengobati vertigo berkepanjanganselamat malam ...
4,AD-73,imunisasi dapat dan polio,imunisasi poliomalam dok anak umur tahun perm...
...,...,...,...
2922,KD-43404,keluhan setelah berhubungan,keluhan berhubungandok minggu bersetubuh pasan...
2923,KD-43406,ingin menambah tinggi badan,menambah tinggi badanselamat malam dok obats...
2924,KD-43411,anak 1 tahun boleh dikasih nasi ?,anak tahun dikasih nasi bayi berumur tahun men...
2925,KD-43446,mencegah infeksi virus zika,mencegah infeksi virus zikaselamat siang dok b...


In [77]:
testing_duplicate = testing.iloc[:,[0]]

In [78]:
pred_res_bidan_testing = classifier_svc_bidan.predict(res_testing)


In [79]:
pred_res_dalam_testing = classifier_svc_dalam.predict(res_testing)


In [80]:
pred_res_anak_testing = classifier_svc_anak.predict(res_testing)


In [81]:
pred_res_kulit_testing = classifier_svc_kulit.predict(res_testing)


In [82]:
pred_res_gizi_testing = classifier_svc_gizi.predict(res_testing)


In [83]:
pred_res_THT_testing = classifier_svc_THT.predict(res_testing)


In [84]:
pred_res_gigi_testing = classifier_svc_gigi.predict(res_testing)


In [85]:
pred_res_mata_testing = classifier_svc_mata.predict(res_testing)


In [86]:
pred_res_bedah_testing = classifier_svc_bedah.predict(res_testing)


In [87]:
pred_res_jiwa_testing = classifier_svc_jiwa.predict(res_testing)


In [88]:
pred_res_tulang_testing = classifier_svc_tulang.predict(res_testing)


In [89]:
pred_res_jantung_testing = classifier_svc_jantung.predict(res_testing)


In [90]:
pred_res_urologi_testing = classifier_svc_urologi.predict(res_testing)


In [91]:
pred_res_saraf_testing = classifier_svc_saraf.predict(res_testing)


In [92]:
pred_res_paru_testing = classifier_svc_paru.predict(res_testing)


In [93]:
pred_res_umum_testing = classifier_svc_umum.predict(res_testing)


In [94]:
testing_duplicate['Kebidanan'] = pred_res_bidan_testing
testing_duplicate['Dalam'] = pred_res_dalam_testing
testing_duplicate['Anak'] = pred_res_anak_testing
testing_duplicate['Kulit'] = pred_res_kulit_testing
testing_duplicate['Gizi'] = pred_res_gizi_testing
testing_duplicate['THT'] = pred_res_THT_testing

testing_duplicate['Gigi'] = pred_res_gigi_testing
testing_duplicate['Mata'] = pred_res_mata_testing
testing_duplicate['Bedah'] = pred_res_bedah_testing
testing_duplicate['KesehatanJiwa'] = pred_res_jiwa_testing

testing_duplicate['Tulang'] = pred_res_tulang_testing
testing_duplicate['Jantung'] = pred_res_jantung_testing
testing_duplicate['Urologi'] = pred_res_urologi_testing
testing_duplicate['Saraf'] = pred_res_saraf_testing
testing_duplicate['Paru'] = pred_res_paru_testing

testing_duplicate['Umum'] = pred_res_umum_testing

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [95]:
testing_duplicate.to_csv('/content/drive/MyDrive/Data TK 1 Text Mining/datakedua.csv', index=False, header=False)

In [96]:
arrayAccuracyScore = []
arrayF1Score = []
arrayTestLabel = [test_label_bidan,test_label_dalam,test_label_anak,test_label_kulit,test_label_gizi,test_label_THT,test_label_gigi,test_label_mata,test_label_bedah,test_label_jiwa,test_label_tulang,test_label_jantung,test_label_urologi,test_label_saraf,test_label_paru,test_label_umum]
arrayPredRes = [pred_res_bidan,pred_res_dalam,pred_res_anak,pred_res_kulit,pred_res_gizi,pred_res_THT,pred_res_gigi,pred_res_mata,pred_res_bedah,pred_res_jiwa,pred_res_tulang,pred_res_jantung,pred_res_urologi,pred_res_saraf,pred_res_paru,pred_res_umum]

for f, b in zip(arrayTestLabel, arrayPredRes):
    acc = accuracy_score(f, b)
    arrayAccuracyScore.append(acc)
    f1 = f1_score(f, b, average='macro')
    arrayF1Score.append(f1)
from statistics import mean

print('Prediksi SVC')
print('Akurasi:', mean(arrayAccuracyScore))
print('F1:', mean(arrayF1Score))

Prediksi SVC
Akurasi: 0.728997975708502
F1: 0.6095600603671112
